In [1]:
import pandas as pd
import numpy as np

In [9]:
all_data = pd.read_csv('gamelvl_joined.csv')
all_data.head()

,Unnamed: 0,year,game_ID,home_numberGame,home_ID_team,home_isB2B,home_isB2BFirst,home_isB2BSecond,home_name,home_countDaysRest,...,away_points,away_plusminus,away_ind_Win,away_ind_SecondB2B,away_ind_overtime,away_offensive_rating,away_defensive_rating,away_RAPTOR_offensive,away_RAPTOR_defensive,away_RAPTOR_net
0,1,2018,21700002,1,1610612744,False,True,False,GSW,120,...,122,1,1,0,0,114.7,106.1,6.1,2.5,8.7
1,2,2018,21700001,1,1610612739,False,True,False,CLE,120,...,99,-3,0,0,0,107.6,103.9,-1.0,4.7,3.7
2,3,2018,21700010,1,1610612762,False,True,False,UTA,120,...,96,-10,0,0,0,112.5,111.0,3.9,-2.3,1.5
3,4,2018,21700011,1,1610612759,False,False,False,SAS,120,...,99,-8,0,0,0,113.4,111.1,4.8,-2.4,2.3
4,5,2018,21700007,2,1610612738,True,False,True,BOS,0,...,108,8,1,0,0,109.8,110.1,1.2,-1.5,-0.3


In [12]:
#Engineering some new variables that may be useful in predicting spread of NBA games

all_data["spread"] = all_data["home_points"] - all_data["away_points"]
all_data["home_ftr"] = all_data["home_freethrow_attempts"] / all_data["home_fieldgoal_attempts"]
all_data["away_ftr"] = all_data["away_freethrow_attempts"] / all_data["away_fieldgoal_attempts"]
all_data["home_pace"] = (all_data["home_fieldgoal_attempts"] + .44 * all_data["home_freethrow_attempts"] - all_data["home_off_reb"]+all_data["home_turnovers"])/48
all_data["away_pace"] = (all_data["away_fieldgoal_attempts"] + .44 * all_data["away_freethrow_attempts"] - all_data["away_off_reb"]+all_data["away_turnovers"])/48
all_data["home_turnover_rate"] = all_data["home_turnovers"] / (all_data["home_fieldgoal_attempts"] + .44 * all_data["home_freethrow_attempts"] - all_data["home_off_reb"]+all_data["home_turnovers"])
all_data["away_turnover_rate"] = all_data["away_turnovers"] / (all_data["away_fieldgoal_attempts"] + .44 * all_data["away_freethrow_attempts"] - all_data["away_off_reb"]+all_data["away_turnovers"])
all_data["home_atr"] = all_data["home_assists"] / all_data["home_turnovers"]
all_data["away_atr"] = all_data["away_assists"] / all_data["away_turnovers"]
all_data.head()

,Unnamed: 0,year,game_ID,home_numberGame,home_ID_team,home_isB2B,home_isB2BFirst,home_isB2BSecond,home_name,home_countDaysRest,...,away_RAPTOR_net,spread,home_ftr,away_ftr,home_pace,away_pace,home_turnover_rate,away_turnover_rate,home_atr,away_atr
0,1,2018,21700002,1,1610612744,False,True,False,GSW,120,...,8.7,-1,0.262500,0.195876,2.088333,2.257500,0.169593,0.119970,2.000000,2.153846
1,2,2018,21700001,1,1610612739,False,True,False,CLE,120,...,3.7,3,0.301205,0.284091,2.125000,2.125000,0.166667,0.117647,1.117647,2.000000
2,3,2018,21700010,1,1610612762,False,True,False,UTA,120,...,1.5,10,0.197531,0.155844,2.021667,2.005833,0.154575,0.218114,1.866667,1.000000
3,4,2018,21700011,1,1610612759,False,False,False,SAS,120,...,2.3,8,0.213483,0.270588,2.007500,2.023333,0.114155,0.133855,2.000000,1.769231
4,5,2018,21700007,2,1610612738,True,False,True,BOS,0,...,-0.3,-8,0.230769,0.394737,2.171667,2.045833,0.143899,0.152749,1.533333,1.266667


In [51]:
def get_averages(year:int):

    all_data_2024 = all_data[all_data['year'] == year]

    # Group by home_name (home team) and calculate the mean across numeric variables, removing NA values
    grouped_by_home = all_data_2024.groupby('home_name').mean()

    # Extract column names containing "home"
    columns_containing_home = [col for col in grouped_by_home.columns if 'home' in col]

    # Select only columns containing "home" to isolate averages when teams are playing at home
    average_values_by_home_team = grouped_by_home[columns_containing_home].reset_index()

    # Repeating but for away
    average_values_by_away_team = all_data_2024.groupby('away_name').mean().dropna()
    away_columns = [col for col in average_values_by_away_team.columns if 'away' in col]
    average_values_by_away_team = average_values_by_away_team[away_columns].reset_index()

    joined_averages = pd.merge(average_values_by_away_team, average_values_by_home_team, how='cross')
    joined_averages = joined_averages[joined_averages['away_name'] != joined_averages['home_name']]
    joined_averages['matchup'] = joined_averages['away_name'] + ' @ ' + joined_averages['home_name']

    # Drop unnecessary columns
    joined_averages = joined_averages.drop(columns=['away_name', 'home_name'])
    return joined_averages

get_averages(2024)


,away_numberGame,away_ID_team,away_isB2B,away_isB2BFirst,away_isB2BSecond,away_countDaysRest,away_countDaysNextGame,away_isWin,away_fieldgoal_made,away_fieldgoal_attempts,...,home_offensive_rating,home_defensive_rating,home_RAPTOR_offensive,home_RAPTOR_defensive,home_RAPTOR_net,home_ftr,home_pace,home_turnover_rate,home_atr,matchup
1,36.162162,1.610613e+09,0.324324,0.135135,0.081081,4.351351,1.189189,0.405405,41.810811,92.270270,...,113.7,116.4,-1.8,-0.9,-2.7,0.234412,2.078794,0.123157,2.307505,ATL @ BKN
2,36.162162,1.610613e+09,0.324324,0.135135,0.081081,4.351351,1.189189,0.405405,41.810811,92.270270,...,123.3,111.7,7.9,3.8,11.6,0.225298,2.095625,0.119240,2.445766,ATL @ BOS
3,36.162162,1.610613e+09,0.324324,0.135135,0.081081,4.351351,1.189189,0.405405,41.810811,92.270270,...,109.0,119.9,-6.4,-4.4,-10.9,0.225172,2.079342,0.133990,2.071946,ATL @ CHA
4,36.162162,1.610613e+09,0.324324,0.135135,0.081081,4.351351,1.189189,0.405405,41.810811,92.270270,...,114.6,116.3,-0.8,-0.8,-1.7,0.240381,2.094466,0.128236,2.131202,ATL @ CHI
5,36.162162,1.610613e+09,0.324324,0.135135,0.081081,4.351351,1.189189,0.405405,41.810811,92.270270,...,115.3,112.4,-0.2,3.1,2.9,0.224026,2.069539,0.134035,2.364976,ATL @ CLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,35.000000,1.610613e+09,0.368421,0.184211,0.131579,4.342105,1.210526,0.210526,42.684211,92.631579,...,108.9,117.9,-6.6,-2.4,-9.0,0.240264,2.096820,0.146045,1.641533,WAS @ POR
895,35.000000,1.610613e+09,0.368421,0.184211,0.131579,4.342105,1.210526,0.210526,42.684211,92.631579,...,117.1,115.9,1.6,-0.4,1.2,0.246741,2.150768,0.126110,2.522138,WAS @ SAC
896,35.000000,1.610613e+09,0.368421,0.184211,0.131579,4.342105,1.210526,0.210526,42.684211,92.631579,...,110.3,117.3,-5.1,-1.8,-6.9,0.230536,2.162741,0.146028,2.087578,WAS @ SAS
897,35.000000,1.610613e+09,0.368421,0.184211,0.131579,4.342105,1.210526,0.210526,42.684211,92.631579,...,112.9,118.6,-2.6,-3.1,-5.7,0.234390,2.107457,0.138769,2.180957,WAS @ TOR
